# Length-Adaptive Sequential Recommendation

**Hybrid SASRec + LightGCN with Adaptive Fusion on MovieLens-1M**

---

## 🚀 Quick Start

This notebook is ready to run! Just:
1. **Enable GPU** (recommended): Settings → Accelerator → GPU T4
2. **Click "Run All"** or run cells sequentially

**Expected Time:**
- With GPU T4: ~8-10 minutes per model
- With CPU: ~40-50 minutes per model

All data is already preprocessed and included in the repository!

## Step 1: Clone Repository

Cloning from: https://github.com/faroukq1/length-adaptive.git

In [ ]:
# Clone repository with all preprocessed data
!git clone https://github.com/faroukq1/length-adaptive.git

# Change to project directory
%cd length-adaptive

# Verify structure
!echo "✓ Source code:"
!ls -la src/

!echo "\n✓ Preprocessed data:"
!ls -lh data/ml-1m/processed/

!echo "\n✓ Co-occurrence graph:"
!ls -lh data/graphs/

## Step 2: Install Dependencies

Installing PyTorch Geometric and other required packages

In [ ]:
# Install required packages quietly
!pip install -q torch-geometric tqdm scikit-learn pandas matplotlib

print("✓ All dependencies installed successfully!")

## Step 3: Verify GPU Setup

Check if GPU is available and will be used for training

In [ ]:
# Check GPU availability
!python check_gpu.py

## Step 4: Quick Test (2 epochs)

Verify everything works with a quick 2-epoch test on both SASRec and Hybrid models

In [ ]:
# Run quick training test
!python test_training.py

## Step 5: Train Hybrid Model (50 epochs)

Train our best model: **Hybrid with Discrete Fusion**

This model adapts based on user history length:
- Short history users (≤10 items): More collaborative filtering (GNN)
- Medium users (10-50 items): Balanced fusion
- Long history users (>50 items): More sequential patterns (Transformer)

In [ ]:
# Train Hybrid model with discrete fusion
!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 50 \
    --batch_size 256 \
    --lr 0.001 \
    --d_model 64 \
    --n_heads 2 \
    --n_blocks 2 \
    --patience 10

## Step 6: Train SASRec Baseline (Optional)

Train baseline model for comparison

In [ ]:
# Train SASRec baseline
!python experiments/run_experiment.py \
    --model sasrec \
    --epochs 50 \
    --batch_size 256 \
    --lr 0.001 \
    --patience 10

## Step 7: Train All Models (Optional - takes 3-5 hours)

Uncomment to train all 5 model variants:
- `sasrec`: Transformer baseline
- `hybrid_fixed`: Fixed fusion weight (α=0.5)
- `hybrid_discrete`: Bin-based fusion (our approach)
- `hybrid_learnable`: Per-user learned weights
- `hybrid_continuous`: Neural network fusion

In [ ]:
# Uncomment to run all experiments
# !bash scripts/run_all_experiments.sh

## Step 8: Analyze Results

Generate comparison tables and visualizations

In [ ]:
# Generate analysis
!python experiments/analyze_results.py --save_csv

## Step 9: Display Results

Show performance comparison table

In [ ]:
import pandas as pd
import os

# Load overall comparison
if os.path.exists('results/overall_comparison.csv'):
    df = pd.read_csv('results/overall_comparison.csv')
    
    print("\n" + "="*80)
    print("📊 OVERALL PERFORMANCE")
    print("="*80 + "\n")
    print(df.to_string(index=False))
    
    # Highlight best model
    best = df.iloc[0]
    print("\n" + "="*80)
    print(f"🏆 BEST MODEL: {best['Model']}")
    print("="*80)
    print(f"  NDCG@10: {best['NDCG@10']:.4f}")
    print(f"  HR@10:   {best['HR@10']:.4f}")
    print(f"  MRR@10:  {best['MRR@10']:.4f}")
    print("="*80 + "\n")
    
    # Show improvement over baseline
    sasrec_row = df[df['Model'] == 'sasrec']
    if not sasrec_row.empty:
        sasrec_ndcg = sasrec_row.iloc[0]['NDCG@10']
        hybrid_ndcg = best['NDCG@10']
        improvement = ((hybrid_ndcg - sasrec_ndcg) / sasrec_ndcg) * 100
        print(f"📈 Improvement over SASRec baseline: {improvement:+.2f}%\n")
else:
    print("❌ No results found. Run experiments first!")

## Step 10: Performance by User Group

Compare performance across different user history lengths

In [ ]:
import glob

# Load grouped metrics
print("\n" + "="*80)
print("📊 PERFORMANCE BY USER GROUP")
print("="*80 + "\n")

groups = ['short', 'medium', 'long']
group_files = {
    'short': 'results/comparison_short.csv',
    'medium': 'results/comparison_medium.csv',
    'long': 'results/comparison_long.csv'
}

for group_name, filepath in group_files.items():
    if os.path.exists(filepath):
        df_group = pd.read_csv(filepath)
        print(f"\n{group_name.upper()} HISTORY USERS:")
        print("-" * 80)
        print(df_group.to_string(index=False))
        print()

## Step 11: Visualize Learning Curves

Plot training loss and validation NDCG over epochs

In [ ]:
import json
import matplotlib.pyplot as plt

# Find all experiment results
result_folders = glob.glob('results/*_*')

if len(result_folders) > 0:
    plt.figure(figsize=(14, 5))
    
    # Plot 1: Training Loss
    plt.subplot(1, 2, 1)
    for folder in result_folders:
        history_path = os.path.join(folder, 'history.json')
        if os.path.exists(history_path):
            with open(history_path, 'r') as f:
                history = json.load(f)
            
            # Extract model name from folder
            parts = os.path.basename(folder).split('_')
            model_name = '_'.join(parts[:-2]) if len(parts) > 2 else parts[0]
            
            plt.plot(history['train_loss'], label=model_name, marker='o', markersize=3, linewidth=2)
    
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('BPR Loss', fontsize=12)
    plt.title('Training Loss', fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Validation NDCG@10
    plt.subplot(1, 2, 2)
    for folder in result_folders:
        history_path = os.path.join(folder, 'history.json')
        if os.path.exists(history_path):
            with open(history_path, 'r') as f:
                history = json.load(f)
            
            parts = os.path.basename(folder).split('_')
            model_name = '_'.join(parts[:-2]) if len(parts) > 2 else parts[0]
            
            ndcg_values = [m['ndcg@10'] for m in history['val_metrics']]
            plt.plot(ndcg_values, label=model_name, marker='o', markersize=3, linewidth=2)
    
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('NDCG@10', fontsize=12)
    plt.title('Validation NDCG@10', fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/learning_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Saved to: results/learning_curves.png")
else:
    print("No results to plot. Run experiments first!")

## Step 12: Download Results

Create a zip file of all results for download

In [ ]:
# Create zip of all results
!zip -r results.zip results/

print("\n✅ Success!")
print("Download 'results.zip' from the Output tab (right sidebar) →")
print("\nContains:")
print("  • Model checkpoints (best_model.pt)")
print("  • Training history (history.json)")
print("  • Test metrics (results.json)")
print("  • Comparison tables (CSV files)")
print("  • Learning curves (PNG)")

---

## ✅ Summary

You've successfully:
1. ✅ Cloned repository with preprocessed data
2. ✅ Installed all dependencies
3. ✅ Tested training pipeline
4. ✅ Trained recommendation models
5. ✅ Analyzed and compared results
6. ✅ Visualized learning curves

---

## 🔬 Key Results

**Dataset:** MovieLens-1M
- 6,034 users
- 3,533 items  
- 1M+ ratings
- 74,170 co-occurrence edges

**Models Trained:**
- SASRec (Transformer baseline)
- Hybrid with Discrete Fusion (length-adaptive)

**Metrics:** Hit Rate (HR), NDCG, MRR at K={5, 10, 20}

---

## 🚀 Next Steps

**Experiment with hyperparameters:**
```python
!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 100 \
    --batch_size 512 \
    --lr 0.0005 \
    --d_model 128 \
    --n_heads 4
```

**Try different fusion strategies:**
- `--model hybrid_learnable` - Per-user learned weights
- `--model hybrid_continuous` - Neural network fusion

**Analyze specific user groups:**
Check `results/comparison_*.csv` for performance on short/medium/long history users

---

## 📚 Resources

- **GitHub:** https://github.com/faroukq1/length-adaptive
- **Paper:** Length-Adaptive Hybrid Sequential Recommendation
- **Dataset:** MovieLens-1M (GroupLens)

---

**Questions or issues?** Check the README.md and EXPERIMENTS.md in the repository.